In [2]:
import pandas as pd

In [3]:
reviews = pd.read_json('C:/Users/jyzho/Downloads/reviews_1500k.json', lines=True)

ValueError: Expected object or value

In [ ]:
reviews

In [18]:
# Note that this takes a while to run
reviews['text'] = reviews['text'].str.lower().str.split()

### Removing stop words

In [19]:
# Download set of stop words
import nltk
from nltk.corpus import stopwords
# nltk.download('stopwords')
stop_words_set = set(stopwords.words('english'))

In [20]:
# Removing stop words
reviews['text'] = reviews['text'].apply(lambda x: [item for item in x if item not in stop_words_set])

In [21]:
' '.join(reviews['text'][0])

'total bill horrible service? $8gs. crooks actually nerve charge us $69 3 pills. checked online pills 19 cents each! avoid hospital ers costs.'

In [22]:
# Convert back to string, takes a while to run
reviews['text'] = reviews['text'].apply(lambda x: [' '.join(x)][0])

### Removing unnecessary characters

In [ ]:
# Remove unnecessary characters here
reviews['text'] = reviews['text'].apply(lambda x: [x.replace("*", "").\
                                                   replace("#", "").\
                                                   replace("-", "")][0])

### Lemmatization and stemming

In [ ]:
# Lemmatization and stemming
from nltk.stem import PorterStemmer

ps = PorterStemmer()
reviews['lemmatized_stemmed_text'] = reviews['text'].apply(lambda x: [ps.stem(x)][0])



### Fix contractions

In [ ]:
import re
from contractions import contractions_dict
nltk.download('punkt')
# Code retrieved from this page: https://medium.com/@pemagrg/pre-processing-text-in-python-ad13ea544dae
def expand_contractions(text, contractions_dict):
    contractions_pattern = re.compile('({})'.format('|'.join(contractions_dict.keys())),
                                      flags=re.IGNORECASE | re.DOTALL)
    
    def expand_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = contractions_dict.get(match) \
            if contractions_dict.get(match) \
            else contractions_dict.get(match.lower())
        expanded_contraction = expanded_contraction
        return expanded_contraction
    
    expanded_text = contractions_pattern.sub(expand_match, text)
    expanded_text = re.sub("'", "", expanded_text)
    return expanded_text
    
reviews['lemmatized_stemmed_text'] = reviews['lemmatized_stemmed_text'].\
                                        apply(lambda x: [expand_contractions(x,contractions_dict)][0])


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jyzho\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Remove punctuation

In [1]:
# Remove all punctuation
reviews['text'] = reviews['text'].apply(lambda x: x.translate(string.maketrans("",""), string.punctuation))

NameError: name 'reviews' is not defined